<a href="https://colab.research.google.com/github/ProfessorPatrickSlatraigh/CIS9557_bctw/blob/main/cis9557_GROUP_5_SuperStore_timeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CIS9557 SuperStore Time Series  
<b>Scaffolding and Data Wrangling for Group 5</b>  
<b>Using `Ship Date`, and a.) `Category`, or b.) `Region` Features</b>  
  
*example of data wrangling for time-series, forecasting*  



##0. Housekeeping  

Import the usual suspects

In [5]:
import pandas as pd

In [6]:
import numpy as np

In [4]:
from datetime import datetime

In [7]:
from matplotlib import pyplot as plt

Load (US) sample Superstore.XLS Orders spreadsheet from CSV file on Professor Patrick's Github  


In [1]:
!curl 'https://raw.githubusercontent.com/ProfessorPatrickSlatraigh/data/main/USsampleSuperstoreOrders_May-2024.csv' -o usOrders.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2234k  100 2234k    0     0  4475k      0 --:--:-- --:--:-- --:--:-- 4478k




---



##1. Create a Date Range Scaffold  
  
Spin up a series of `timedate` values to use as scaffolding for time series analysis  


In [8]:
def timescaffold(startIncludes='01-Jan-2020', endIncludes='31-Mar-2020', periodFrequency='daily'):
    """

    Args:
      startIncludes:   the scaffold series starting date in %d-%b-$Y (dd-Mmm-YYYY) format
      endIncludes:     the scaffold series ending date in %d-%b-$Y (dd-Mmm-YYYY) format
      periodFrequency: the frequency of the scaffold series periods (daily, weekdays, businessdays, weekly, monthly, annually)
    """

    print(f'From {startIncludes} to {endIncludes} at a {periodFrequency} rate.\n')

    # set a pattern for the expected date format in strings (user entry)
    date_format = '%d-%b-%Y'  # %d for day, %b for abbreviated month, %Y for four-digit year
    # test for a missing start date parameter and prompt for a valid value
    # if startIncludes == '01-Jan-2020':
    goodStart = input(f'\nThe time series will start with {startIncludes}. Is that correct? ')
    while goodStart.lower()[0] != 'y':
        startIncludes = input('What value would you like to start with? ')
        try:
            startScaffold = datetime.strptime(startIncludes, date_format)
            goodStart='yes'
        except:
            print('Your input was not a valid date in the format "dd-Mmm-YYYY". Enter a new value.')
    startScaffold = datetime.strptime(startIncludes, date_format)
    print(f'Thank you.  The scaffold time series will begin with {startScaffold}.')
    # test for a missing end date parameter and prompt for a valid value
    # if endIncludes == '31-Mar-2020':
    goodEnd = input(f'\nThe time series will end with {endIncludes}. Is that correct? ')
    while goodEnd.lower()[0] != 'y':
        endIncludes = input('What value would you like to start with?')
        try:
            endScaffold = datetime.strptime(endIncludes, date_format)
            goodEnd = 'yes'
        except:
            print('Your input was not a valid date in the format "dd-Mmm-YYYY". Enter a new value. ')
    endScaffold = datetime.strptime(endIncludes, date_format)
    print(f'Thank you.  The scaffold time series will end with {datetime.strptime(endIncludes, date_format)}.')

    return(pd.date_range(start=startScaffold, end=endScaffold, freq='D'))



In [9]:
scaffold = timescaffold()

From 01-Jan-2020 to 31-Mar-2020 at a daily rate.


The time series will start with 01-Jan-2020. Is that correct? y
Thank you.  The scaffold time series will begin with 2020-01-01 00:00:00.

The time series will end with 31-Mar-2020. Is that correct? y
Thank you.  The scaffold time series will end with 2020-03-31 00:00:00.


In [ ]:
print(len(scaffold))

In [ ]:
print(scaffold)



---



##2. Load Profit Dataframe  
  
Create a `Dataframe` of the `ShippedDate` and `Profit` variables (columns) from the `Orders` dataset (sheet) retaining the following features:  
- `Row ID`  
- `Ship Date`  
- `Segment`  
- `State`  
- `Region`  
- `Product ID`  
- `Category`  
- `Profit` (target)  
  

Create a list of the columns to keep and the names to use for them

In [10]:
keep_columns_ls = ['Row ID', 'Ship Date', 'Category', 'Segment', 'State', 'Region', 'Product ID', 'Category', 'Profit']

In [11]:
orders_df = pd.read_csv('usOrders.csv', usecols=keep_columns_ls)

In [ ]:
orders_df.columns

In [ ]:
orders_df.describe()

Create a `datetime` value on each row from its  'Ship Date'

In [12]:
orders_df['DateShipped'] = pd.to_datetime(orders_df['Ship Date'])

In [ ]:
orders_df.head(15)

##3. Merge Timedate Scaffold and Superstore Dataframe  

Merge the time series scaffold with the dataset in the `dataframe`  

In [13]:
scaffold_df = pd.DataFrame(scaffold, columns=['Date'])

In [ ]:
scaffold_df.head(15)

In [ ]:
len(scaffold_df)

###3.a) Enrich Scaffolding for `Ship Date` and `Region`  
  
<font color=blue><i>Note: the steps in the section are to scaffold the Superstore dataset for `Ship Date` and `Region` and produce a dataset that has at least one row (record) for every combination of `Ship Date` and `Region`.  If the original dataset has more than one row for any combination of `Ship Date` and `Region` those will be aggregated so that we have only one row for every combination of desired features -- that one row will have the aggregate profit for that combination of features.</i></font>  
  
<font color=red><i>If you intend to analyze the dataset by `Ship Date` and `Category`, then use <b>Step 3.b)</b> instead of this step.</i></font>



Create a `regions_df` `dataframe` with a value for every `Region`

In [ ]:
# DataFrame with the 'Region' values
regions = ['West', 'East', 'Central', 'South']
regions_df = pd.DataFrame({'Region': regions})

In [ ]:
# Performing a cross join to combine every 'Date' with every 'Region'
expandedScaffold_df = scaffold_df.assign(key=1).merge(regions_df.assign(key=1), on='key').drop('key', axis=1)

In [ ]:
expandedScaffold_df.head(10)

In [ ]:
len(expandedScaffold_df)

In [ ]:
training_df = pd.merge(expandedScaffold_df, orders_df, left_on=['Date', 'Region'], right_on=['DateShipped', 'Region'], how='outer').drop('DateShipped', axis=1)

Now we have an enriched dataset with all the `Date Shipped` and `Region` combinations included at least once, but some of these record may have missing values (NaN) for `Profit`.  

In [ ]:
training_df.head(20)

In [ ]:
len(training_df)

Sub-total the `Profit` data in the `training_df` dataset by `Region` and `Date`

In [ ]:
# Use a specific column for more meaningful count, here using 'Value' column
summaryProfit_df = training_df.groupby(['Region', 'Date'])['Profit'].count().reset_index(name='Count')

In [ ]:
len(summaryProfit_df)

In [ ]:
# Display the result
print(summaryProfit_df)

Sub-total the `Date` instances in the `training_df` dataset by `Region` and `Date`

In [ ]:
# Use a specific column for more meaningful count, here using 'Value' column
summaryDate_df = training_df.groupby(['Region', 'Date'])['Date'].count().reset_index(name='Count')

In [ ]:
len(summaryDate_df)

In [ ]:
# Display the result
print(summaryDate_df)

<center><b><font color=blue>NOW PROCEED TO STEP #4</b></font></center>  




---



###3.b) Enrich Scaffolding for `Ship Date` and `Category`  
  
<font color=blue><i>Note: the steps in the section are to scaffold the Superstore dataset for `Ship Date` and `Category` and produce a dataset that has at least one row (record) for every combination of `Ship Date` and `Category`.  If the original dataset has more than one row for any combination of `Ship Date` and `Category` those will be aggregated so that we have only one row for every combination of desired features -- that one row will have the aggregate profit for that combination of features.</i></font>  
  
<font color=red><i>If you intend to analyze the dataset by `Ship Date` and `Region`, then use <b>Step 3.a)</b> instead of this step.</i></font>



Create a `categories_df` `dataframe` with a value for every `Category`

In [14]:
# DataFrame with the 'Category' values
categories = ['Furniture', 'Office Supplies', 'Technology']
categories_df = pd.DataFrame({'Category': categories})

In [15]:
# Performing a cross join to combine every 'Date' with every 'Catgeory'
expandedScaffold_df = scaffold_df.assign(key=1).merge(categories_df.assign(key=1), on='key').drop('key', axis=1)

In [ ]:
expandedScaffold_df.head(10)

In [ ]:
len(expandedScaffold_df)

In [23]:
training_df = pd.merge(expandedScaffold_df, orders_df, left_on=['Date', 'Category'], right_on=['DateShipped', 'Category'], how='left').drop('DateShipped', axis=1)

Now we have an enriched dataset with all the `Date Shipped` and `Category` combinations included at least once, but some of these record may have missing values (NaN) for `Profit`.  

In [ ]:
training_df.head(20)

In [26]:
training_df.tail(20)

,Date,Category,Row ID,Ship Date,Segment,State,Region,Product ID,Profit
421,2020-03-27,Technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422,2020-03-28,Furniture,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423,2020-03-28,Office Supplies,9867.0,3/28/2020,Corporate,New York,East,OFF-ST-10002615,282.2092
424,2020-03-28,Office Supplies,9868.0,3/28/2020,Corporate,New York,East,OFF-AP-10000595,3.4086
425,2020-03-28,Technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426,2020-03-29,Furniture,1061.0,3/29/2020,Consumer,Texas,Central,FUR-CH-10003396,-29.2524
427,2020-03-29,Office Supplies,NaN,NaN,NaN,NaN,NaN,NaN,NaN
428,2020-03-29,Technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429,2020-03-30,Furniture,9858.0,3/30/2020,Corporate,Washington,West,FUR-TA-10001889,610.8624
430,2020-03-30,Office Supplies,2558.0,3/30/2020,Corporate,Florida,South,OFF-AP-10002287,0.5244


In [ ]:
len(training_df)

Sub-total the `Profit` data in the `training_df` dataset by `Category` and `Date`

In [22]:
# Use a specific column for more meaningful count, here using 'Value' column
summaryProfit_df = training_df.groupby(['Category', 'Date'])['Profit'].count().reset_index(name='Count')

In [23]:
len(summaryProfit_df)

273

In [ ]:
# Display the result
print(summaryProfit_df)

Sub-total the `Date` instances in the `training_df` dataset by `Category` and `Date`

In [25]:
# Use a specific column for more meaningful count, here using 'Value' column
summaryDate_df = training_df.groupby(['Category', 'Date'])['Date'].count().reset_index(name='Count')

In [ ]:
len(summaryDate_df)

In [ ]:
# Display the result
print(summaryDate_df)



---



<center><b><font color=blue>Regardless of whether sub-step 3.a or 3.b above was used, all of the following steps apply.</b></font></center>



---



##4. Fill Missing Values

We can replace missing (NaN) values for `Profit` with zeroes.  Those will typically occur on the new, scaffolder rows we created for feature combinations with `Ship Date` that did not occur in the original dataset.

In [27]:
len(training_df)

441

In [ ]:
training_df.columns

In [ ]:
training_df.describe()

In [28]:
training_df['Profit'].fillna(0, inplace=True)



---



##5. Write CSV File

We write the resulting file as `training.csv` to the current working directory.  

In [29]:
training_df.to_csv('training.csv', sep=',', header=True, mode='w', encoding='utf-8')



---

